In [1]:
pip install dotenv


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3.13 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
"""
USDA NASS QuickStats API - Agricultural Labor Data Fetcher
Fetches hired farm labor data and exports to CSV.

Usage:
    from fetch_labor_data import fetch_and_save_labor_data
    fetch_and_save_labor_data(2017)
"""

import os
import requests
import csv
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://quickstats.nass.usda.gov/api/api_GET/"

# Map output column names to QuickStats short_desc values
LABOR_CATEGORIES = [
    ("Hired farm labor", "LABOR, HIRED - NUMBER OF WORKERS"),
    ("Workers by days worked\n- 150 days or more", "LABOR, HIRED, GE 150 DAYS WORKED - NUMBER OF WORKERS"),
    ("Workers by days worked\n- Less than 150 days", "LABOR, HIRED, LT 150 DAYS WORKED - NUMBER OF WORKERS"),
    ("Reported only workers working \n150 days or more", "LABOR, HIRED, ONLY GE 150 DAYS WORKED - NUMBER OF WORKERS"),
    ("Reported only workers working\n less than 150 days", "LABOR, HIRED, ONLY LT 150 DAYS WORKED - NUMBER OF WORKERS"),
    ("Reported both - workers working\n 150 days or more and workers\n working less than 150 days\n- 150 days or more, workers", 
     "LABOR, HIRED, BOTH GE 150 & LT 150 DAYS WORKED, GE 150 DAYS - NUMBER OF WORKERS"),
    ("Reported both - workers working\n 150 days or more and workers\n working less than 150 days\n- less than 150 days, workers",
     "LABOR, HIRED, BOTH GE 150 & LT 150 DAYS WORKED, LT 150 DAYS - NUMBER OF WORKERS"),
    ("Total migrant workers", "LABOR, MIGRANT - NUMBER OF WORKERS"),
    ("Unpaid workers", "LABOR, UNPAID - NUMBER OF WORKERS"),
]


def _fetch_category(year: int, short_desc: str) -> dict:
    """Fetch data for a single labor category. Returns {state: value}."""
    params = {
        "key": API_KEY,
        "source_desc": "CENSUS",
        "sector_desc": "ECONOMICS",
        "commodity_desc": "LABOR",
        "short_desc": short_desc,
        "domain_desc": "TOTAL",
        "agg_level_desc": "STATE",
        "year": year,
        "format": "JSON",
    }
    
    resp = requests.get(BASE_URL, params=params)
    resp.raise_for_status()
    data = resp.json()
    
    result = {}
    for rec in data.get("data", []):
        state = rec.get("state_name", "").upper()
        if state and state not in ("US TOTAL", "OTHER STATES"):
            result[state] = rec.get("Value", "")
    return result


def fetch_and_save_labor_data(year: int, output_path: str = None) -> str:
    """
    Fetch agricultural labor data for a given census year and save to CSV.
    
    Args:
        year: Census year (e.g., 2017, 2022)
        output_path: Optional output file path. Defaults to 'agricultural_labor_{year}.csv'
    
    Returns:
        Path to the saved CSV file
    """
    if not API_KEY:
        raise ValueError("API_KEY not found. Add API_KEY=your_key to .env file")
    
    if output_path is None:
        output_path = f"agricultural_labor_{year}.csv"
    
    print(f"Fetching labor data for {year}...")
    
    # Collect data by state
    states_data = {}
    
    for col_name, short_desc in LABOR_CATEGORIES:
        print(f"  Fetching: {col_name.split(chr(10))[0]}...")
        try:
            category_data = _fetch_category(year, short_desc)
            for state, value in category_data.items():
                if state not in states_data:
                    states_data[state] = {}
                states_data[state][col_name] = value
        except requests.exceptions.RequestException as e:
            print(f"    Warning: Failed - {e}")
    
    # Build CSV rows
    headers = ["state", "year"] + [col for col, _ in LABOR_CATEGORIES]
    rows = []
    
    for state in sorted(states_data.keys()):
        row = [state, year]
        for col_name, _ in LABOR_CATEGORIES:
            row.append(states_data[state].get(col_name, ""))
        rows.append(row)
    
    # Write CSV
    with open(output_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(headers)
        writer.writerows(rows)
    
    print(f"Saved {len(rows)} states to {output_path}")
    return output_path


In [2]:
fetch_and_save_labor_data(2012)

Fetching labor data for 2012...
  Fetching: Hired farm labor...
  Fetching: Workers by days worked...
  Fetching: Workers by days worked...
  Fetching: Reported only workers working ...
  Fetching: Reported only workers working...
  Fetching: Reported both - workers working...
  Fetching: Reported both - workers working...
  Fetching: Total migrant workers...
  Fetching: Unpaid workers...
Saved 50 states to agricultural_labor_2012.csv


'agricultural_labor_2012.csv'